In [1]:
# SPDX-License-Identifier: Apache-2.0 AND CC-BY-NC-4.0
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Quantum Enhanced Optimization for Radar and Communications Applications 


The Low Autocorrelation Binary Sequences (LABS) is an important and challenging optimization problem with applications related to radar, telecommunications, and other signal related applications. This CUDA-Q Academic module will focus on a clever quantum-enhanced hybrid method developed in a collaboration between Kipu Quantum, University of the Basque Country EHU, and NVIDIA for solving the LABS problem. (This notebook was jointly developed with input from all collaborators.)

Other CUDA-Q Academic modules like [Divide and Conquer MaxCut QAOA](https://github.com/NVIDIA/cuda-q-academic/tree/main/qaoa-for-max-cut) and [Quantum Finance](https://github.com/NVIDIA/cuda-q-academic/blob/main/quantum-applications-to-finance/03_qchop.ipynb), demonstrate how quantum computing can be used outright to solve optimization problems. This notebook demonstrates a slightly different approach. Rather than considering QPUs as the tool to produce the final answer, it demonstrates how quantum can be used to enhance the effectiveness of leading classical methods.  

The benefits of such an approach were highlighted in [Scaling advantage with quantum-enhanced memetic tabu search for LABS](https://arxiv.org/html/2511.04553v1).  This notebook, co-created with the authors of the paper, will allow you to explore the findings of their research and write your own CUDA-Q code that builds a representative quantum-enhanced workflow for solving the LABS problem. Moreover, it will introduce advancements in counteradiabatic optimization techniques on which reduce the quantum resources required to run on a QPU.

**Prerequisites:** This lab assumes you have a basic knowledge of quantum computing, including operators, gates, etc.  For a refresher on some of these topics, explore the [Quick start to Quantum](https://github.com/NVIDIA/cuda-q-academic/tree/main/quick-start-to-quantum) series.

**In this lab you will:**
* 1. Understand the LABS problem and its relation ot radar and communication applications.
* 2. Solve LABS classically with memetic tabu search and learn about the limitations of such methods.
* 3. Code a couteradiabatic algorithm using CUDA-Q to produce approximate solutions to the LABS problem.
* 4. Use the CUDA-Q results to seed your tabu search and understand the potential benefits of this approach.


**Terminology you will use:**
* Low autocorrelation of binary sequences (LABS)
* counteradiabatic optimization
* memetic-tabu search

**CUDA-Q Syntax you will use:**
* cudaq.sample()
* @cudaq.kernel
* ry(), rx(), rz(), x(), h() 
* x.ctrl()

Run the code below to initialize the libraries you will need.

In [2]:
import cudaq
import numpy as np
from math import floor
import auxiliary_files.labs_utils as utils

## The LABS problem and applications

The **Low Autocorrelation Binary Sequences (LABS)** problem is fundamental to many applications, but originated with applications to radar. 

Consider a radar that monitors airport traffic.  The radar signal sent to detect incoming planes must have as much range as possible to ensure safe approaches are planned well in advance.  The range of a radar signal can be increased by sending a longer pulse.  However, in order to differentiate between multiple objects, pulses need to be short to provide high resolution. So, how do you handle situations where you need both?

One solution is a technique called pulse compression.  The idea is to send a long signal, but vary the phase at regular intervals such that the resolution is increased. Generally, the initial signal will encode a binary sequence of phase shifts, where each interval corresponds to a signal with a 0 or 180 degree phase shift. 

The tricky part is selecting an optimal encoding sequence.  When the signal returns, it is fed into a matched filter with the hope that a singular sharp peak will appear, indicating clear detection.  The autocorrelation of the original signal, or how similar the signal is to itself,  determines if a single peak or a messier signal with sidelobes will be detected. A signal should have high autocorrelation when overlayed on top of itself, but low autocorrelation when shifted with a lag. 

Consider the image below.  The signal on the left has a crisp single peak while the single on the right produces many undesirable sidelobes which can inhibit clear detection.  

<img src="images/quantum_enhanced_optimization_LABS/radar.png" width="800">


So, how do you select a good signal?   This is where LABS comes in, defining these questions as a binary optimization problem. Given a binary sequence of length $N$, $(s_1 \cdots s_N) \in {\pm 1}^N$, the goal is to minimize the following objective function.

$$ E(s) = \sum_{k=1}^{N-1} C_k^2 $$

Where $C_k$ is defined as. 

 $$C_k= \sum_{i=1}^{N-k} s_is_{i+k}$$


So, each $C_k$ computes how similar the original signal is to the shifted one for each offset value $k$.  To explore this more, try the interactive widget linked [here](https://nvidia.github.io/cuda-q-academic/interactive_widgets/labs_visualization.html).  See if you can select a very good and very poor sequence and see the difference for the case of $N=7$.

## Classical Solution of the LABS problem

The LABS problem is tricky to solve for a few reasons. First, the configuration space grows exponentially.  Second, underlying symmetries of the problem result in many degeneracies in the optimization landscape severely inhibiting local search methods. 

<div style="background-color: #f9fff0; border-left: 6px solid #76b900; padding: 15px; border-radius: 4px; box-shadow: 0px 2px 4px rgba(0,0,0,0.1);">
    <h3 style="color: #76b900; margin-top: 0; margin-bottom: 10px;">Exercise 1:</h3>
    <p style="font-size: 16px; color: #333;">
Using the widget above, try to find some of the symmetries for the LABS problem. That is, for a fixed bitstring length, can you find patterns to produce the same energy with different pulse patterns. 
</div>

The best known performance for a classical optimization technique is Memetic Tabu search (MTS) which exhibits a scaling of $O(1.34^N)$.  The MTS algorithm is depicted below.  It begins with a randomly selected population of bitstrings and finds the best solution from them.  Then, a child is selected by sampling directly from or combining multiple bitstrings from the population.  The child is mutated with probability $p_{mutate}$ and then input to a tabu search, which performs a modified greedy local search starting from the child bitstring.  If the result is better than the best in the population, it is updated as the new leader and randomly replaces a  bitstring in the population.


<img src="images/quantum_enhanced_optimization_LABS/mts_algorithm.png" width="500">

Such an approach is fast, parallelizable, and allows for exploration with improved searching of the solution landscape.  

<div style="background-color: #f9fff0; border-left: 6px solid #76b900; padding: 15px; border-radius: 4px; box-shadow: 0px 2px 4px rgba(0,0,0,0.1);">
    <h3 style="color: #76b900; margin-top: 0; margin-bottom: 10px;">Exercise 2:</h3>
    <p style="font-size: 16px; color: #333;">
Before exploring any quantum approach, get a sense for how MTS works by coding it yourself based generally on the figure above. Algorithms for the combine and mutate steps are provided below as used in the paper. You may need to research more specific details of the process, especially for how tabu search is performed. The MTS procedure should output and optimal bitstring and its energy.  Also, write a function to visualize the results including the energy distribution of the final population.
</div>



<img src="images/quantum_enhanced_optimization_LABS/combine_mutate.png" width="400">



In [3]:
#TODO - Write code to perform MTS
import numpy as np
import random
import matplotlib.pyplot as plt
def labs_energy(s):
    """
    Compute LABS energy E(s) for s in {+1, -1}^N
    """
    N = len(s)
    E = 0
    for k in range(1, N):
        Ck = np.sum(s[:N-k] * s[k:])
        E += Ck**2
    return E
def combine(p1, p2):
    """
    One-point crossover
    """
    N = len(p1)
    k = random.randint(1, N-1)
    return np.concatenate([p1[:k], p2[k:]])

def mutate(s, p_mut):
    """
    Flip each bit with probability p_mut
    """
    s = s.copy()
    for i in range(len(s)):
        if random.random() < p_mut:
            s[i] *= -1
    return s
def tabu_search(s0, tenure=5, max_steps=100):
    """
    Single-bit-flip tabu search starting from s0
    """
    N = len(s0)
    s = s0.copy()
    best = s.copy()
    best_E = labs_energy(best)

    tabu = np.zeros(N, dtype=int)  # tabu expiration times
    t = 0

    for step in range(max_steps):
        t += 1
        E_current = labs_energy(s)

        best_move = None
        best_move_E = float("inf")

        for i in range(N):
            s_trial = s.copy()
            s_trial[i] *= -1
            E_trial = labs_energy(s_trial)

            # Aspiration criterion
            if tabu[i] > t and E_trial >= best_E:
                continue

            if E_trial < best_move_E:
                best_move_E = E_trial
                best_move = i

        if best_move is None:
            break

        # Apply move
        s[best_move] *= -1
        tabu[best_move] = t + tenure

        if best_move_E < best_E:
            best = s.copy()
            best_E = best_move_E

    return best
def memetic_tabu_search(
    N,
    pop_size=20,
    p_mut=0.02,
    tabu_tenure=5,
    tabu_steps=100,
    iterations=500
):
    """
    Classical Memetic Tabu Search for LABS
    """
    # Initialize population
    population = [np.random.choice([-1, 1], size=N) for _ in range(pop_size)]
    energies = [labs_energy(s) for s in population]

    best_idx = np.argmin(energies)
    best = population[best_idx].copy()
    best_E = energies[best_idx]

    for it in range(iterations):
        # Make child
        if random.random() < 0.5:
            child = population[random.randrange(pop_size)]
        else:
            p1, p2 = random.sample(population, 2)
            child = combine(p1, p2)

        # Mutate
        child = mutate(child, p_mut)

        # Tabu search refinement
        refined = tabu_search(child, tabu_tenure, tabu_steps)
        E_refined = labs_energy(refined)

        # Update best
        if E_refined < best_E:
            best = refined.copy()
            best_E = E_refined

        # Replace random population member if better
        replace_idx = random.randrange(pop_size)
        if E_refined < energies[replace_idx]:
            population[replace_idx] = refined
            energies[replace_idx] = E_refined

    return best, best_E, population
N = 5
best_s, best_E, final_pop = memetic_tabu_search(
    N,
    pop_size=30,
    iterations=700
)

print("Best energy:", best_E)



Best energy: 2


## Building a Quantum Enhanced Workflow

Despite the effectiveness of MTS, it still exhibits exponential scaling  $O(1.34^N)$ behavior and becomes intractable for large $N$.  Quantum computing provides a potential alternative method for solving the LABS problem because the properties of entanglement, interference, and superpositon may allow for a better global search.  Recent demonstrations have even produced evidence that the quantum approximate optimization algorithm (QAOA) can be used to reduce the scaling of the LABS problem to $O(1.21^N)$ for $N$ between 28 and 40 with quantum minimum finding.

However, current quantum hardware limitations restrict solution to problems of greater than about $N=20$, meaning that it will be some time before quantum approaches can outperform the classical state of the art. It should also be noted that standard QAOA can struggle with LABS and require many layers to converge the parameters if other tricks are not employed.

The authors of [Scaling advantage with quantum-enhanced memetic tabu search for LABS](https://arxiv.org/html/2511.04553v1) cleverly explored an alternate path that combines quantum and classical approaches and might be able to provide a more near-term benefit.  Instead of expecting the quantum computer to solve the problem entirely, they asked how a quantum approach might enhance MTS.

The basic idea is that a quantum optimization routine could run first and the resulting state be sampled to produce a better population for MTS. Many such heuristics for defining the initial population are possible, but the rest of this notebook will explore their methodology, help you to build the workflow yourself, and allow you to analyze the benefits of their approach.

The first step of quantum enhanced MTS (QE-MTS) is to prepare a circuit with CUDA-Q that approximates the ground state of the Hamiltonian corresponding to the LABS problem. You could do this with any optimization algorithm such as QAOA or using an adiabatic approach.  (See the [Quantum Portfolio Optimization](https://github.com/NVIDIA/cuda-q-academic/blob/main/quantum-applications-to-finance/03_qchop.ipynb) CUDA-Q Academic lab for a detailed comparison of these two common approaches.)

The authors of this work opted for an adiabatic approach (More on why later). Recall that the goal of an adiabatic optimization is to begin with a Hamiltonian that has an easily prepared ground state ($H_i$). Then, the adiabatic Hamiltonian $H_{ad}$ can be constructed as $H_{ad}(\lambda) = (1-\lambda)H_i +\lambda H_f $, where $\lambda$ is a function of time and $H_f$ is the Hamiltonian representing a qubit encoding of the LABS problem. 

$$H_f = 2 \sum_{i=1}^{N-2} \sigma_i^z \sum_{k=1}^{\lfloor \frac{N-i}{2} \rfloor} \sigma_{i+k}^z 
+ 4 \sum_{i=1}^{N-3} \sigma_i^z \sum_{t=1}^{\lfloor \frac{N-i-1}{2} \rfloor} \sum_{k=t+1}^{N-i-t} \sigma_{i+t}^z \sigma_{i+k}^z \sigma_{i+k+t}^z$$

The authors also selected $H_i = \sum_i h^x_i \sigma^x_i $ which has an easily prepared ground state of $\ket{+}^{\otimes N}$.

The challenge for implementing the optimization procedure becomes selection of an operator that will quickly and accurately evolve to the ground state of $H_f$.  One approach is to use a so-called auxiliary countradiabatic (CD) term $H_{CD}$, which corrects diabatic transitions that jump out of the ground state during the evolution. The figure below demonstrates the benefit of using a CD correction.


<img src="images/quantum_enhanced_optimization_LABS/counteradiabatic.png" width="900">




An operator called the adiabatic gauge potential $A_{\lambda}$ is the ideal choice for the CD term as it suppresses all possible diabatic transitions, resulting in the following total system to evolve.

$$ H(\lambda) = H_{ad}(\lambda) + \lambda H_{CD} (\lambda) $$

$A(\lambda)$ is derrived from $H_{ad}(\lambda)$  (see paper for details) as it contains information about underlying physics of the problem. 

There is a problem though.  The $A(\lambda)$ term cannot be efficiently expressed exactly and needs to be approximated.  It also turns out that in the so called impulse regime, where the adiabatic evolution is very fast, $H_{cd} (\lambda)$ dominates $H_{ad}(\lambda)$, meaning that the final implementation corresponds to the operator $H(\lambda) = H^1_{cd}(\lambda)$ where  $H^1_{cd}(\lambda)$ is a first order approximation of $A(\lambda)$ (see equation 7 in the paper).

A final step is to use Trotterization to define the quantum circuit to apply $e^{-\theta (t) i H_{cd}}$. The details for this derivation are shown in the appendix of the paper. and result from equation B3 is shown below.  

$$
\begin{equation}
\begin{aligned}
U(0, T) = \prod_{n=1}^{n_{\text{trot}}} & \left[ \prod_{i=1}^{N-2} \prod_{k=1}^{\lfloor \frac{N-i}{2} \rfloor} R_{Y_i Z_{i+k}}\big(4\theta(n\Delta t)h_i^x\big) R_{Z_i Y_{i+k}}\big(4\theta(n\Delta t)h_{i+k}^x\big) \right] \\
& \times \prod_{i=1}^{N-3} \prod_{t=1}^{\lfloor \frac{N-i-1}{2} \rfloor} \prod_{k=t+1}^{N-i-t} \bigg( R_{Y_i Z_{i+t} Z_{i+k} Z_{i+k+t}}\big(8\theta(n\Delta t)h_i^x\big) \\
& \quad \times R_{Z_i Y_{i+t} Z_{i+k} Z_{i+k+t}}\big(8\theta(n\Delta t)h_{i+t}^x\big) \\
& \quad \times R_{Z_i Z_{i+t} Y_{i+k} Z_{i+k+t}}\big(8\theta(n\Delta t)h_{i+k}^x\big) \\
& \quad \times R_{Z_i Z_{i+t} Z_{i+k} Y_{i+k+t}}\big(8\theta(n\Delta t)h_{i+k+t}^x\big) \bigg)
\end{aligned}
\end{equation}
$$

It turns out that this implementation is more efficient than QAOA in terms of gate count. The authors calculated that for $N=67$, QAOA would require 1.4 million entangling gates while the CD approach derived here requires only 236 thousand entangling gates.


<div style="background-color: #f9fff0; border-left: 6px solid #76b900; padding: 15px; border-radius: 4px; box-shadow: 0px 2px 4px rgba(0,0,0,0.1);">
    <h3 style="color: #76b900; margin-top: 0; margin-bottom: 10px;">Exercise 3:</h3>
    <p style="font-size: 16px; color: #333;">
At first glance, this equation might looks quite complicated. However, observe the structure and note two "blocks" of terms.  Can you spot them?  

They are 2 qubit terms that look like $R_{YZ}(\theta)$ or $R_{ZY}(\theta)$.

As well as 4 qubit terms that look like $R_{YZZZ}(\theta)$, $R_{ZYZZ}(\theta)$, $R_{ZZYZ}(\theta)$, or $R_{ZZZY}(\theta)$.

Thankfully the authors derive a pair of circuit implementations for the two and four qubit terms respectively, shown in the figures below.

Using CUDA-Q, write a kernel for each which will be used later to construct the full implementation.

* Hint: Remember that the adjoint of a rotation gate is the same as rotating in the opposite direction. 

* Hint: You may also want to define a CUDA-Q kernel for an R$_{ZZ}$ gate.

* Hint: Implementing a circuit from a paper is a great place where AI can help accelerate your work.  If you have access to a coding assistant, feel free to use it here.
</div>

<img src="images/quantum_enhanced_optimization_LABS/kernels.png" width="1300">


In [4]:


# TODO  Write CUDA-Q kernels to apply the 2 and 4 qubit operators. 

# import cudaq
# import math

@cudaq.kernel
def rzz(theta: float, q0: cudaq.qubit, q1: cudaq.qubit):
    cx(q0, q1)
    rz(theta, q1)
    cx(q0, q1)

@cudaq.kernel
def two_qubit_rotations(q0: cudaq.qubit, q1: cudaq.qubit, theta: float):
    rx(math.pi / 2, q1)
    rzz(theta, q0, q1)
    rx(math.pi / 2, q0)
    rx(-math.pi / 2, q1)
    rzz(theta, q0, q1)
    rx(-math.pi / 2, q0)

@cudaq.kernel
def four_qubit_rotations(q0: cudaq.qubit, q1: cudaq.qubit,q2: cudaq.qubit,q3: cudaq.qubit , theta: float):

    rx(-math.pi / 2, q0)
    ry(math.pi / 2, q1)
    ry(-math.pi / 2, q2)
    rzz(-math.pi / 2, q0, q1)
    rzz(-math.pi / 2, q2, q3)
    rx(math.pi / 2, q0)
    ry(-math.pi / 2, q1)
    ry(math.pi / 2, q2)
    rx(-math.pi / 2, q3)
    rx(-math.pi / 2, q1)    
    rx(-math.pi / 2, q2)
    rzz(theta, q1, q2)
    rx(math.pi / 2, q1)    
    rx(math.pi, q2)
    ry(math.pi / 2, q1)    
    rzz(math.pi / 2, q0, q1)
    rx(math.pi / 2, q0)    
    ry(-math.pi / 2, q1)
    rzz(-theta, q1, q2)
    rx(math.pi / 2, q1)   
    rx(-math.pi, q2)    
    rzz(-theta, q1, q2)
    rx(-math.pi, q1) 
    ry(math.pi / 2, q2)
    rzz(-math.pi / 2, q2, q3)
    ry(-math.pi / 2, q2)
    rx(-math.pi / 2, q3)
    rx(-math.pi / 2, q2)
    rzz(theta, q1, q2)
    rx(math.pi / 2, q1)   
    rx(math.pi / 2, q2)  
    ry(-math.pi / 2, q1)   
    ry(math.pi / 2, q2)  
    rzz(math.pi / 2, q0, q1)
    rzz(math.pi / 2, q2, q3)
    ry(math.pi / 2, q1)   
    ry(-math.pi / 2, q2) 
    rx(math.pi / 2, q3)  

'''
#test

@cudaq.kernel
def kernel_to_draw():
    q = cudaq.qvector(4)

    theta = 0  # arbitrary parameter


    two_qubit_rotations(q[0], q[1], theta)

print(cudaq.draw(kernel_to_draw))
'''






'\n#test\n\n@cudaq.kernel\ndef kernel_to_draw():\n    q = cudaq.qvector(4)\n\n    theta = 0  # arbitrary parameter\n\n\n    two_qubit_rotations(q[0], q[1], theta)\n\nprint(cudaq.draw(kernel_to_draw))\n'

There are a few additional items we need to consider before completing the final implementation of the entire circuit.  One simplification we can make is that for our problem the $h_i^x$ terms are all 1 and any $h_b^x$ terms are 0, and are only there for generalizations of this model. 

The remaining challenge is derivation of the angles that are used to apply each of the circuits you defined above. These are obtained from two terms $\lambda(t)$ and $\alpha(t)$.  


The $\lambda(t)$ defines an annealing schedule and is generally a Sin function which slowly "turns on" the problem Hamiltonian.  For computing our angles, we need the derivative of $\lambda(t)$.

The $\alpha$ term is a bit trickier and is the solution to a set of differential equations which minimize the distance between $H^1_{CD}(\lambda)$ and $A(\lambda)$.  The result is 

$$\alpha(t) = \frac{-\Gamma_1(t)}{\Gamma_2(t)} $$

Where $\Gamma_1(t)$ and $\Gamma_2(t)$ are defined in equations 16 and 17 of the paper and essentially depend on the structure of the optimization problem.  Curious learners can look at the functions in `labs_utils.py`  to see how these are computed, based on the problem size and specific time step in the Trotter process. 


<div style="background-color: #f9fff0; border-left: 6px solid #76b900; padding: 15px; border-radius: 4px; box-shadow: 0px 2px 4px rgba(0,0,0,0.1);">
    <h3 style="color: #76b900; margin-top: 0; margin-bottom: 10px;">Exercise 4:</h3>
    <p style="font-size: 16px; color: #333;">
The `compute_theta` function, called in the following cells, requires all indices of the two and four body terms. These will be used again in our main kernel to apply the respective gates.  Use the products in the formula below to finish the function in the cell below.  Save them as `G2` and `G4` where each is a list of lists of indices defining the two and four term interactions. As you are translating an equation to a set of loops, this is a great opportunity to use an AI coding assistant.

$$
\begin{equation}
\begin{aligned}
U(0, T) = \prod_{n=1}^{n_{\text{trot}}} & \left[ \prod_{i=1}^{N-2} \prod_{k=1}^{\lfloor \frac{N-i}{2} \rfloor} R_{Y_i Z_{i+k}}\big(4\theta(n\Delta t)h_i^x\big) R_{Z_i Y_{i+k}}\big(4\theta(n\Delta t)h_{i+k}^x\big) \right] \\
& \times \prod_{i=1}^{N-3} \prod_{t=1}^{\lfloor \frac{N-i-1}{2} \rfloor} \prod_{k=t+1}^{N-i-t} \bigg( R_{Y_i Z_{i+t} Z_{i+k} Z_{i+k+t}}\big(8\theta(n\Delta t)h_i^x\big) \\
& \quad \times R_{Z_i Y_{i+t} Z_{i+k} Z_{i+k+t}}\big(8\theta(n\Delta t)h_{i+t}^x\big) \\
& \quad \times R_{Z_i Z_{i+t} Y_{i+k} Z_{i+k+t}}\big(8\theta(n\Delta t)h_{i+k}^x\big) \\
& \quad \times R_{Z_i Z_{i+t} Z_{i+k} Y_{i+k+t}}\big(8\theta(n\Delta t)h_{i+k+t}^x\big) \bigg)
\end{aligned}
\end{equation}
$$

</div>


In [5]:
def get_interactions(N):
    """
    Generates the interaction sets G2 and G4 based on the loop limits in Eq. 15.
    Returns standard 0-based indices as lists of lists of ints.
    
    Args:
        N (int): Sequence length.
        
    Returns:
        G2: List of lists containing two body term indices
        G4: List of lists containing four body term indices
    """
    
    G2 = []
    G4 = []

    # ---- Two-body terms ----

    for i in range(0, N - 2):  # 0..N-3 inclusive
        max_k = (N - i - 1) // 2
        for k in range(1, max_k + 1):
            G2.append([i, i + k])

    # ---- Four-body terms ----

    for i in range(0, N - 3):  # 0..N-4 inclusive
        max_t = (N - i - 2) // 2
        for t in range(1, max_t + 1):
            k_max = N - i - t - 1
            for k in range(t + 1, k_max + 1):
                G4.append([i, i + t, i + k, i + k + t])
                
    return G2, G4
#test
#print(get_interactions(10))




<div style="background-color: #f9fff0; border-left: 6px solid #76b900; padding: 15px; border-radius: 4px; box-shadow: 0px 2px 4px rgba(0,0,0,0.1);">
    <h3 style="color: #76b900; margin-top: 0; margin-bottom: 10px;">Exercise 5:</h3>
    <p style="font-size: 16px; color: #333;">
You are now ready to construct the entire circuit and run the counteradiabatic optimization procedure. The final kernel needs to apply Equation 15 for a specified total evolution time $T$ and the `n_steps` number of Trotter steps.  It must also take as input, the indices for the two and four body terms and the thetas to be applied each step, as these cannot be computed within a CUDA-Q kernel.

The helper function `compute_theta` computes the theta parameters for you, using a few additional functions in accordance with the equations defined in the paper.
</div>


In [6]:
import math
import cudaq


@cudaq.kernel
def trotterized_circuit(N: int,G2: list[list[int]],G4: list[list[int]],steps: int,dt: float,
                        T: float,thetas: list[float]):

    reg = cudaq.qvector(N)
    h(reg)

    # Apply Eq. (15) trotter steps
    for n in range(steps):
        theta_n = thetas[n]
        for pair in G2:
            i = pair[0]
            j = pair[1]
            two_qubit_rotations(reg[i],reg[j], 4.0 * theta_n*dt)
        # ---- 4-qubit blocks: for each [a, b, c, d] ----
        # Apply Y on each one (one at a time), angle 8 theta
        for quad in G4:
            a = quad[0]
            b = quad[1]
            c = quad[2]
            d = quad[3]
            four_qubit_rotations(reg[a],reg[b],reg[c],reg[d], 8.0 * theta_n*dt)
T=1               # total time
n_steps = 1       # number of trotter steps
dt = T / n_steps
N = 11
G2, G4 = get_interactions(N)

thetas =[]

for step in range(1, n_steps + 1):
    t = step * dt
    theta_val = utils.compute_theta(t, dt, T, N, G2, G4)
    thetas.append(theta_val)


# TODO - Sample your kernel to make sure it works
# Create a sample object
shots = 100  # number of times to run the circuit

result = cudaq.sample(trotterized_circuit, N, G2, G4, n_steps, dt, T, thetas, shots_count=shots)

# Print the results
print(result)



{ 00000000001:1 00000001110:1 00000010101:1 00000010111:1 00000011011:1 00000100001:1 00000111111:1 00001100000:1 00001100100:1 00001101100:1 00001110011:1 00001110101:1 00010000111:1 00010010001:1 00010011111:1 00010110011:1 00010111100:1 00011000000:1 00011001101:1 00011110010:1 00011111011:1 00100100100:1 00100101111:1 00100110000:1 00100110100:1 00101011000:1 00101011100:1 00101101110:1 00110000011:1 00110010011:1 00110101011:1 00110101101:1 00111001110:1 00111110011:1 01000001011:1 01000010100:1 01000111011:1 01001010000:1 01001011110:1 01010001000:1 01010011001:1 01011010110:1 01011100101:1 01100001110:1 01100101111:1 01100110110:1 01100111101:1 01101110001:1 01110000110:1 01110010100:1 01110101001:1 01110111011:2 01111010111:1 01111110000:1 10000001111:1 10000010111:1 10000011100:1 10001000110:1 10001001111:1 10010100101:1 10011011111:1 10011110100:1 10100010011:1 10100100110:1 10100111010:1 10100111101:1 10101001111:1 10101011011:1 10110001010:1 10110010101:1 10110111111:1 1011

## Generating Quantum Enhanced Results

Recall that the point of this lab is to demonstrate the potential benefits of running a quantum subroutine as a preprocessing step for classical optimization of a challenging problem like LABS. you now have all of the tools you need to try this for yourself.

<div style="background-color: #f9fff0; border-left: 6px solid #76b900; padding: 15px; border-radius: 4px; box-shadow: 0px 2px 4px rgba(0,0,0,0.1);">
    <h3 style="color: #76b900; margin-top: 0; margin-bottom: 10px;">Exercise 6:</h3>
    <p style="font-size: 16px; color: #333;">
Use your CUDA-Q code to prepare an initial population for your memetic search algorithm and see if you can improve the results relative to a random initial population.  If you are running on a CPU, you will need to run smaller problem instances. The code below sets up the problem

</div>


In [7]:
import numpy as np
import cudaq
import random

# -----------------------------
# Helper: get counts from cudaq.sample
# -----------------------------
def _counts(res):
    if hasattr(res, "items"):
        return dict(res.items())
    if hasattr(res, "counts"):
        return dict(res.counts())
    return dict(res)

def bitstring_to_spins(bitstr: str) -> np.ndarray:
    # '0' -> +1, '1' -> -1
    return np.array([+1 if b == "0" else -1 for b in bitstr], dtype=int)

# -----------------------------
# Seeded MTS = your Ex2 MTS with ONLY init changed
# -----------------------------
def memetic_tabu_search_seeded(
    N,
    init_population,          # list of np.ndarray (spins)
    pop_size=20,
    p_mut=0.02,
    tabu_tenure=5,
    tabu_steps=100,
    iterations=500
):
    # Initialize population from quantum seeds, pad with random if needed
    population = [np.array(s, dtype=int).copy() for s in init_population[:pop_size]]
    while len(population) < pop_size:
        population.append(np.random.choice([-1, 1], size=N))

    energies = [labs_energy(s) for s in population]

    best_idx = np.argmin(energies)
    best = population[best_idx].copy()
    best_E = energies[best_idx]

    for it in range(iterations):
        if random.random() < 0.5:
            child = population[random.randrange(pop_size)]
        else:
            p1, p2 = random.sample(population, 2)
            child = combine(p1, p2)

        child = mutate(child, p_mut)

        refined = tabu_search(child, tabu_tenure, tabu_steps)
        E_refined = labs_energy(refined)

        if E_refined < best_E:
            best = refined.copy()
            best_E = E_refined

        replace_idx = random.randrange(pop_size)
        if E_refined < energies[replace_idx]:
            population[replace_idx] = refined
            energies[replace_idx] = E_refined

    return best, best_E, population

# -----------------------------
# EX6: quantum seeds -> MTS
# -----------------------------
T = 1
n_steps = 1
dt = T / n_steps
N = 11
G2, G4 = get_interactions(N)

thetas = []
for step in range(1, n_steps + 1):
    t = step * dt
    thetas.append(utils.compute_theta(t, dt, T, N, G2, G4))

# 1) sample Ex5 circuit
shots = 300     # use >10 for diversity
res = cudaq.sample(trotterized_circuit, N, G2, G4, n_steps, dt, T, thetas, shots_count=shots)

# 2) build seed population from sampled bitstrings (best energies first)
ranked = []
for bitstr, c in _counts(res).items():
    s = bitstring_to_spins(bitstr)
    ranked.append((labs_energy(s), -c, s))  # (energy, -count, spins)

# sort by energy first, then by count
ranked.sort(key=lambda x: (x[0], x[1]))


pop_size = 30
seed_pop = [s for (_, _, s) in ranked[:pop_size]]
while len(seed_pop) < pop_size:
    seed_pop.append(np.random.choice([-1, 1], size=N))

print("Best quantum seed energy:", ranked[0][0] if ranked else None)
print("Most probable bitstring:", res.most_probable())

# 3) baseline: random-init Ex2 MTS
best_r, bestE_r, final_pop_r = memetic_tabu_search(N, pop_size=pop_size, iterations=1000)

# 4) quantum-seeded MTS
best_q, bestE_q, final_pop_q = memetic_tabu_search_seeded(N, seed_pop, pop_size=pop_size, iterations=1000)

print("=== EX6 RESULTS ===")
print("Random-init MTS best energy:   ", bestE_r)
print("Quantum-seeded MTS best energy:", bestE_q)


Best quantum seed energy: 5
Most probable bitstring: 00110001110
=== EX6 RESULTS ===
Random-init MTS best energy:    5
Quantum-seeded MTS best energy: 5


The results clearly show that a population sampled from CUDA-Q results in an improved distribution and a lower energy final result. This is exactly the goal of quantum enhanced optimization.  To not necessarily solve the problem, but improve the effectiveness of state-of-the-art classical approaches. 

A few major caveats need to be mentioned here. First, We are comparing a quantum generated population to a random sample.  It is quite likely that other classical or quantum heuristics could be used to produce an initial population that might even beat the counteradiabatic method you used, so we cannot make any claims that this is the best. 

Recall that the point of the counteradiabatic approach derived in the paper is that it is more efficient in terms of two-qubit gates relative to QAOA. The benefits of this regime would only truly come into play in a setting (e.g. larger problem instance) where it is too difficult to produce a good initial population with any know classical heuristic, and the counteradiabatic approach is more efficiently run on a QPU compared to alternatives.

We should also note that we are comparing a single sample of each approach.  Maybe the quantum sample got lucky or the randomly generated population was unlucky and a more rigorous comparison would need to repeat the analysis many times to draw any confidently conclusions.  

The authors of the paper discuss all of these considerations, but propose an analysis that is quite interesting related to the scaling of the technique. Rather than run large simulations ourselves, examine their results below. 


<img src="images/quantum_enhanced_optimization_LABS/tabu_search_results.png" width="900">

The authors computed replicate median (median of solving the problem repeated with same setup) time to solutions (excluding time to sample from QPU) for problem sizes $N=27$ to $N=37$. Two interesting conclusions can be drawn from this. First, standard memetic tabu search (MTS) is generally faster than quantum enhanced (QE) MTS.  But there are two promising trends. For larger problems, the QE-MTS experiments occasionally have excellent performance with times to solution much smaller than all of the MTS data points.  These outliers indicate there are certain instances where QE-MTS could provide much faster time-to-solution. 

More importantly, if a line of best fit is calculated using the median of each set of medians, the slope of the QE-MTS line is smaller than the MTS!  This seems to indicate that QE solution of this problem scales $O(1.24^N)$ which is better than the best known classical heuristic ($O(1.34^N)$) and the best known quantum approach (QAOA - $O(1.46^N)$).

For problems of size of $N=47$ or greater, the authors anticipate that QE-MTS could be a promising technique and produce good initial populations that are difficult to obtain classically. 

The study reinforces the potential of hybrid workflows enhanced by quantum data such that a classical routine is still the primary solver, but quantum computers make it much more effective.  Future work can explore improvements to both the quantum and classical sides, such as including GPU accelerated memetic search on the classical side.

## Self-validation To Be Completed for Phase 1

We used the table from the reference[3] "T. Packebusch and S. Mertens, Low autocorrelation binary se- quences, Journal of Physics A: Mathematical and Theoretical 49, 165001 (2016)." in "Scaling advantage with quantum-enhanced memetic tabu search for LABS" to check our result is correct or not. 
Also, we calculate the analitycal result for N=2.

We use N = 5 to check the MTS. And use N = 11 to check the QE-MTS. The symmetry is verify.